In [30]:
import requests

# Definições iniciais
base_url = "https://my.timetrade.com/AccountManager/user/getUserList"
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "pt-BR,pt;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "cookie": "JSESSIONID=01E6EF254BC9782FC9D296CDB326D944; psCookie=%7B%22ident%22%3A%22nYn7Crpw9pvPAc4B8VUdLQ%3D%3D%22%2C%22email%22%3A%22megan.halverson%40thomsonreuters.com%22%2C%22rememberMe%22%3Afalse%7D; xSession=%7B%22email%22%3A%22Megan.Halverson%40thomsonreuters.com%22%2C%22hash%22%3A%22nYn7Crpw9pvPAc4B8VUdLQ%3D%3D%22%2C%22token%22%3Anull%7D; AWSALB=mmmKfCynttzsdhtMsry1CNsOpBnKH+qRAWo1wAFIe4GH2n3RUvNDJ4gtGGwOm0D4kcD3E5E9efrzIMg4PrCgwiY5TBlw0meTfSclGPUh9JZAbLtIT5mB4Up/MMuJ; AWSALBCORS=mmmKfCynttzsdhtMsry1CNsOpBnKH+qRAWo1wAFIe4GH2n3RUvNDJ4gtGGwOm0D4kcD3E5E9efrzIMg4PrCgwiY5TBlw0meTfSclGPUh9JZAbLtIT5mB4Up/MMuJ",
    "dnt": "1",
    "priority": "u=0, i",
    "referer": "https://my.timetrade.com/AccountManager/user/getUserList?offset=45&max=15&sort=statusid&order=asc",
    "sec-ch-ua": "\"Microsoft Edge\";v=\"131\", \"Chromium\";v=\"131\", \"Not_A Brand\";v=\"24\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"
}

# Variáveis de controle
offset = 0
max_per_page = 15
results = []
max_pages = 2  # Limite de páginas

while offset // max_per_page < max_pages:  # Verifica se o número de páginas não ultrapassou 19
    # Exibe o offset da página atual
    # print(f"Acessando página com offset {offset}...")
    
    # Monta a URL com os parâmetros
    params = {
        "offset": offset,
        "max": max_per_page,
        "sort": "statusid",
        "order": "asc"
    }
    
    # Faz a requisição
    response = requests.get(base_url, headers=headers, params=params)
    
    # Verifica se a resposta é válida
    if response.status_code != 200:
        print(f"Erro ao buscar página com offset {offset}. Status Code: {response.status_code}")
        break
    
    # Processa o conteúdo da resposta
    data = response.text  # Mantém o retorno como texto
    
    if not data:
        print(f"Nenhum dado retornado para offset {offset}.")
        break
    
    # Extrair o conteúdo da variável userData
    start_index = data.find("var userData = {")  # Localiza onde começa a definição de userData
    end_index = data.find("};", start_index)  # Localiza onde termina a definição de userData
    
    if start_index != -1 and end_index != -1:
        user_data = data[start_index:end_index + 1]  # Extrai a definição de userData
        print(user_data)
    else:
        print(f"Dados não encontrados para o offset {offset}.")
    
    # Adiciona à lista de resultados
    results.append(data)
    
    # Incrementa o offset para a próxima página
    offset += max_per_page
    
    # Verifica condição de parada (se não houver mais páginas)
    if len(data) < max_per_page:
        print("Última página alcançada.")
        break

# Resultados finais
print(f"Total de páginas processadas: {len(results)}")


var userData = {
			'providesLockedUsers' : false,
            'items': [
            	
		            
		            {
		                firstname: 			"Aaron",
		                lastname: 			"Blackmore",
		                "pk.emailaddress": 	"aaron.blackmore\u0040thomsonreuters.com",
		                groupName: 			"Abogado Sales",
		            	isuserlocked:		true, 
			            lastAccessed: 		
												"2023-10-04T15:43:10Z"
											,
						admin:				
												false
											,
						usesLicense:		
												0
											,
						verifiedUser:		true,
		                id: 				"8169049"
		            }
            	
		            
		            ,
		            
		            {
		                firstname: 			"Abby",
		                lastname: 			"Reid",
		                "pk.emailaddress": 	"Abby.Reid\u0040thomsonreuters.com",
		                groupName: 			"Field Sales - ONE SLF",
		            	isuserlocked:		true, 
			            lastAccessed: 		
												"2023-